# PPO - 2-way Single Intersection

## Imports

In [1]:
from datetime import datetime, timezone
import os
import subprocess

import pyautogui
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from sumo_rl import SumoEnvironment

from tyre_pm_reward import tyre_pm_reward

c:\Users\Justin Mak\anaconda3\envs\rl\lib\site-packages\gymnasium\envs\registration.py:693: UserWarning: WARN: Overriding environment GymV26Environment-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


## Create Environment and Instantiate Agent

In [2]:
env_params = {
    "net_file": os.path.join("nets","2way-single-intersection","single-intersection.net.xml"),
    "route_file": os.path.join("nets","2way-single-intersection","single-intersection-vhvh.rou.xml"),
    "out_csv_name": os.path.join("outputs","2way-single-intersection","ppo"),
    "num_seconds": 1e5,
    "single_agent": True,
    "reward_fn": tyre_pm_reward,
    "sumo_seed": 42,
}
env = SumoEnvironment(**env_params)

check_env(env)

env = Monitor(env)  # wrap env to know episode reward, length, time

In [3]:
model = PPO(
    "MlpPolicy",
    env,
    learning_rate=5e-3,
    n_steps=256,
    batch_size=256,
    n_epochs=5,
    gamma=0.95,
    gae_lambda=0.99,
    clip_range=0.3,
    ent_coef=0.09,
    vf_coef=0.05,
    max_grad_norm=0.9,
    tensorboard_log=os.path.join("logs","2way-single-intersection","ppo"),
    verbose=1
)

Using cuda device
Wrapping the env in a DummyVecEnv.


In [4]:
# Evaluate untrained random agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1)

print(mean_reward)
print(std_reward)

-14482.264273
0.0


## Train and Evaluate Agent

In [5]:
# Train agent
model.learn(total_timesteps=1e5)

Logging to logs\2way-single-intersection\ppo\PPO_1
----------------------------
| time/              |     |
|    fps             | 45  |
|    iterations      | 1   |
|    time_elapsed    | 5   |
|    total_timesteps | 256 |
----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 49          |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 512         |
| train/                  |             |
|    approx_kl            | 0.037107177 |
|    clip_fraction        | 0.0734      |
|    clip_range           | 0.3         |
|    entropy_loss         | -1.37       |
|    explained_variance   | -0.00211    |
|    learning_rate        | 0.005       |
|    loss                 | 1.12e+03    |
|    n_updates            | 5           |
|    policy_gradient_loss | -0.0255     |
|    value_loss           | 2.28e+04    |
--------------------------------

In [6]:
# Evaluate trained agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1)
print(mean_reward)
print(std_reward)

-4462.316552
0.0


### Save Model

In [7]:
save_time = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")

model.save(os.path.join("saved_models",f"PPO_2way_single_intersection_{save_time}"))

## Render Video

In [8]:
env_params.update({"use_gui": True})
render_env = SumoEnvironment(**env_params)

check_env(render_env)
render_env.close()

render_env = Monitor(render_env)  # wrap env to know episode reward, length, time

In [9]:
model = PPO.load(os.path.join("saved_models",f"PPO_2way_single_intersection_{save_time}"))

obs, info = env.reset()

VID_LENGTH = 750
for i in range(VID_LENGTH):
    actions, _ = model.predict(obs)
    obs, reward, done, truncated, info = env.step(actions)
    env.render()

    im = pyautogui.screenshot(
        os.path.join("renders","2way-single-intersection",f"img{i}.jpg"),
        region=(0, 0, 2560, 1542)
    )

env.close()  # clean up

In [10]:
ffmpeg_time = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")

subprocess.run(["ffmpeg", "-y", "-r", "5", "-i", "renders\\2way-single-intersection\\img%d.jpg", f"videos\\2way_single_intersection_{ffmpeg_time}.mp4"])

CompletedProcess(args=['ffmpeg', '-y', '-r', '5', '-i', 'renders\\2way-single-intersection\\img%d.jpg', 'videos\\2way_single_intersection_20230412_200916.mp4'], returncode=0)

## Clean-up

In [11]:
env.close()